In [2]:
"""
Decision Tree Implementation
This script implements a decision tree algorithm that can handle both:
- Classification (using Gini impurity)
- Regression (using standard deviation reduction)
"""

import pandas as pd
import math
import numpy as np

# =============================================
# CONFIGURATION SECTION
# =============================================

# Algorithm selection - can be "Classification" or "Regression"
algorithm = "Classification"

# =============================================
# DATA LOADING AND VALIDATION
# =============================================

try:
    # Load dataset from CSV file
    df = pd.read_csv("golf4.txt")
except FileNotFoundError:
    raise FileNotFoundError("Dataset file not found. Please check the path.")

# Detect target column (flexible naming)
target_column = None
possible_names = ['Decision', 'Decesion', 'Class', 'Label', 'Target']
for name in possible_names:
    if name in df.columns:
        target_column = name
        break

if target_column is None:
    raise KeyError("Could not identify target column in dataset")

# Store feature metadata (name: dtype)
dataset_features = {col: df[col].dtype for col in df.columns if col != target_column}
num_features = len(dataset_features)

# Validate algorithm choice against target type
if algorithm == "Regression" and df[target_column].dtype == 'object':
    raise ValueError("Regression requires numeric target")
elif algorithm == "Classification" and df[target_column].dtype != 'object':
    print("Warning: Classification with numeric target")

# Auto-switch to regression if target is numeric
if df[target_column].dtype != 'object':
    algorithm = "Regression"
    global_stdev = df[target_column].std(ddof=0)
    print(f"Using Regression (target is numeric). Global stddev: {global_stdev:.2f}")

# =============================================
# CORE FUNCTIONS
# =============================================

def processContinousFeatures(cdf, column_name, entropy):
    """
    Processes continuous/numeric features by finding optimal split threshold
    and converting to categorical bins.

    Args:
        cdf: DataFrame containing the data
        column_name: Name of the numeric column to process
        entropy: Current entropy value (unused in this implementation)

    Returns:
        Modified DataFrame with binned values
    """
    unique_values = sorted(cdf[column_name].unique())
    subset_ginis = []
    subset_red_stdevs = []

    # Evaluate all possible split thresholds
    for i in range(len(unique_values) - 1):
        threshold = unique_values[i]

        # Split data into two subsets
        subset1 = cdf[cdf[column_name] <= threshold]
        subset2 = cdf[cdf[column_name] > threshold]

        # Calculate subset sizes
        subset1_rows = subset1.shape[0]
        subset2_rows = subset2.shape[0]
        total_instances = cdf.shape[0]

        if algorithm == "Classification":
            # Calculate Gini impurity for classification
            gini_subset1 = 1 - sum((subset1[target_column].value_counts()/subset1_rows)**2)
            gini_subset2 = 1 - sum((subset2[target_column].value_counts()/subset2_rows)**2)
            weighted_gini = (subset1_rows/total_instances)*gini_subset1 + (subset2_rows/total_instances)*gini_subset2
            subset_ginis.append(weighted_gini)

        elif algorithm == "Regression":
            # Calculate standard deviation reduction for regression
            subset1_stdev = subset1[target_column].std(ddof=0)
            subset2_stdev = subset2[target_column].std(ddof=0)
            weighted_stdev = (subset1_rows/total_instances)*subset1_stdev + (subset2_rows/total_instances)*subset2_stdev
            reducted_stdev = cdf[target_column].std(ddof=0) - weighted_stdev
            subset_red_stdevs.append(reducted_stdev)

    # Select best threshold
    if algorithm == 'Classification':
        best_idx = subset_ginis.index(min(subset_ginis))
    else:
        best_idx = subset_red_stdevs.index(max(subset_red_stdevs))

    best_threshold = unique_values[best_idx]

    # Convert numeric column to categorical bins
    cdf[column_name] = np.where(
        cdf[column_name] <= best_threshold,
        f"<={best_threshold}",
        f">{best_threshold}"
    )
    return cdf

def calculateEntropy(df):
    """
    Calculates entropy of the target column.
    Returns 0 for regression trees.

    Args:
        df: DataFrame containing the target column

    Returns:
        Entropy value (float)
    """
    if algorithm == 'Regression':
        return 0

    counts = df[target_column].value_counts()
    total = len(df)
    entropy = 0

    for count in counts:
        prob = count / total
        entropy -= prob * math.log(prob, 2)

    return entropy

def findDecision(ddf):
    """
    Identifies the best feature to split on.

    Args:
        ddf: DataFrame containing features and target

    Returns:
        Name of the best feature to split on
    """
    if algorithm == 'Regression':
        stdev = ddf[target_column].std(ddof=0)

    feature_columns = [col for col in ddf.columns if col != target_column]
    ginis = []
    reducted_stdevs = []

    for col in feature_columns:
        temp_df = ddf[[col, target_column]].copy()

        # Process numeric features
        if dataset_features[col] != 'object':
            temp_df = processContinousFeatures(temp_df, col, 0)

        # Calculate split quality
        classes = temp_df[col].value_counts()
        weighted_metric = 0

        for class_val in classes.index:
            subset = temp_df[temp_df[col] == class_val]
            subset_size = len(subset)

            if algorithm == 'Classification':
                # Calculate Gini impurity
                counts = subset[target_column].value_counts()
                gini = 1 - sum((counts/subset_size)**2)
                weighted_metric += (subset_size/len(temp_df)) * gini
            else:
                # Calculate standard deviation reduction
                subset_stdev = subset[target_column].std(ddof=0)
                weighted_metric += (subset_size/len(temp_df)) * subset_stdev

        if algorithm == 'Classification':
            ginis.append(weighted_metric)
        else:
            reducted_stdevs.append(stdev - weighted_metric)

    # Select best feature
    if algorithm == 'Classification':
        best_idx = ginis.index(min(ginis))
    else:
        best_idx = reducted_stdevs.index(max(reducted_stdevs))

    return feature_columns[best_idx]

def formatRule(indent):
    """Helper function for tree visualization indentation"""
    return "    " * indent

def buildDecisionTree(df, root=0):
    """
    Recursively builds and prints the decision tree.

    Args:
        df: Current subset of data
        root: Current depth in the tree (for indentation)
    """
    charForResp = "'" if algorithm == 'Classification' else ""

    # Get remaining features
    features = [col for col in df.columns if col != target_column]
    if not features:
        # No more features - make final decision
        final = (df[target_column].value_counts().idxmax() if algorithm == 'Classification'
                else df[target_column].mean())
        print(formatRule(root) + f"return {charForResp}{final}{charForResp}")
        return

    # Find best split feature
    best_feature = findDecision(df)
    is_numeric = dataset_features[best_feature] != 'object'

    # Process each value of the best feature
    for value in df[best_feature].unique():
        subset = df[df[best_feature] == value].drop(columns=[best_feature])

        # Determine if this is a leaf node
        is_leaf = False
        if len(subset[target_column].unique()) == 1:
            # All samples same class
            decision = subset[target_column].iloc[0]
            is_leaf = True
        elif algorithm == 'Regression' and subset[target_column].std(ddof=0)/global_stdev < 0.4:
            # Regression early stopping
            decision = subset[target_column].mean()
            is_leaf = True
        elif len(subset.columns) == 1:  # Only target remains
            decision = (subset[target_column].value_counts().idxmax() if algorithm == 'Classification'
                       else subset[target_column].mean())
            is_leaf = True

        # Print decision rule
        condition = value if is_numeric else f"'{value}'"
        print(formatRule(root) + f"if {best_feature} == {condition}:")

        if is_leaf:
            print(formatRule(root+1) + f"return {charForResp}{decision}{charForResp}")
        else:
            buildDecisionTree(subset, root+1)

# =============================================
# EXECUTION
# =============================================

if __name__ == "__main__":
    print("\n=== Decision Tree Construction ===")
    print(f"Algorithm: {algorithm}")
    print(f"Target: {target_column}")
    print(f"Features: {list(dataset_features.keys())}\n")

    buildDecisionTree(df)

Using Regression (target is numeric). Global stddev: 9.32

=== Decision Tree Construction ===
Algorithm: Regression
Target: Decision
Features: ['Outlook', 'Temp.', 'Humidity', 'Wind']

if Outlook == 'Sunny':
    if Temp. == 85:
        return 25
    if Temp. == 80:
        return 30
    if Temp. == 72:
        return 35
    if Temp. == 69:
        return 38
    if Temp. == 75:
        return 48
if Outlook == 'Overcast':
    return 46.25
if Outlook == 'Rain':
    if Wind == 'Weak':
        return 47.666666666666664
    if Wind == 'Strong':
        return 26.5
